# 0. Setting

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

main_directory = '/content/drive/MyDrive/Colab Notebooks/Foiegras'

if not os.path.exists(main_directory):
    os.makedirs(main_directory)
    print(f"'{main_directory}' 디렉토리를 생성했습니다.")
else:
    print(f"'{main_directory}' 디렉토리가 이미 존재합니다.")

'/content/drive/MyDrive/Colab Notebooks/Foiegras' 디렉토리가 이미 존재합니다.


In [3]:
os.chdir(main_directory)
print(f"'{os.getcwd()}' 디렉토리에서 작업 중입니다.")

'/content/drive/MyDrive/Colab Notebooks/Foiegras' 디렉토리에서 작업 중입니다.


In [20]:
import requests
import json
import http.client

class ClovaEmbedding:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request('POST', '/testapp/v1/api-tools/embedding/v2/bbdd4fe4d10e4114aa67715014ecf1af', json.dumps(completion_request), headers)
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res['status']['code'] == '20000':
            return res['result']['embedding']
        else:
            return 'Error'

clova_embedding = ClovaEmbedding(
    host='clovastudio.apigw.ntruss.com',
    api_key='MASKED',
    api_key_primary_val = 'MASKED',
    request_id='MASKED'
    )


class ClovaChat:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def execute(self, completion_request):
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'text/event-stream'
        }

        result_data = None

        with requests.post(self._host + '/testapp/v1/chat-completions/HCX-003',
                           headers=headers, json=completion_request, stream=True) as r:
            for line in r.iter_lines():
                if line:
                    decoded_line = line.decode("utf-8")
                    print(decoded_line)
                    if 'seed' in decoded_line:
                        result_data = decoded_line

        return result_data

clova_chat = ClovaChat(
    host='https://clovastudio.stream.ntruss.com',
    api_key='MASKED',
    api_key_primary_val='MASKED',
    request_id='MASKED'
)

# 1. DB

In [5]:
import pandas as pd
import ast

etf_df_fixed = pd.read_excel(os.path.join(main_directory, 'dataset_2/etf_df_fixed.xlsx'), dtype=str)

# str to list

emb_cols = ['sent1_emb', 'sent2_emb', 'sent3_emb', 'sent4_emb', 'sent5_emb', 'sent6_emb', 'sent7_emb']

for i in range(1, 8):
    emb_col = f'sent{i}_emb'
    etf_df_fixed[emb_col] = etf_df_fixed[emb_col].apply(ast.literal_eval)

# risk (int)

def extract_risk_level(risk_str):
    return int(risk_str.split('등급')[0])

etf_df_fixed['risk'] = etf_df_fixed['risk'].str.strip().apply(extract_risk_level)

# 2. User Info

## 1) Option

In [12]:
user_risk_level = 3 # 투자성향 5단계
user_private_pension = False # 개인연금 O
user_retire_pension = True # 퇴직연금 X

## 2) Text Query

In [13]:
user_info = "엔비디아 10% 이상 담은 상품 중에서, 수익률 Top 3는?" # option 3개 전부 풀고(no-filter), k=164*7(max) 로 해야 정확한 답변 가능. 질문도 여럿 나눠야 토큰 제한 회피.

# 3. RAG

In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


def find_top_k_sentences(user_info, etf_df_fixed, k=10):
    user_emb = clova_embedding.execute({"text": user_info})
    user_emb = np.array(user_emb).reshape(1, -1)
    similarities = []

    for idx, row in etf_df_fixed.iterrows():
        for i in range(1, 8):
            sent_emb = row[f'sent{i}_emb']
            sent_emb = np.array(sent_emb).reshape(1, -1)
            sim = cosine_similarity(user_emb, sent_emb)[0][0]
            similarities.append((sim, row[f'sent{i}'], row['ticker'], row['name']))

    similarities = sorted(similarities, key=lambda x: x[0], reverse=True)[:k]
    similarities_df = pd.DataFrame(similarities, columns=['similarity', 'sentence', 'ticker', 'name'])
    return similarities_df

## 1) Candidates

- Text Query와 유사한 Top-K ETF 선정.

In [15]:
candidates_df = find_top_k_sentences(user_info, etf_df_fixed, k=50)
candidates_df

,similarity,sentence,ticker,name
0,0.599650,[주요보유현황] None [업종비중] No._업종_비중(%)_1_기타_59.51_2...,275980,TIGER 글로벌4차산업혁신기술(합성 H)
1,0.598221,[주요보유현황] None [업종비중] No._업종_비중(%)_1_정보기술_66.09...,412770,TIGER 글로벌메타버스액티브
2,0.597703,[주요보유현황] None [업종비중] No._업종_비중(%)_1_정보기술_30.81...,482730,TIGER 미국S&P500+10%프리미엄초단기
3,0.596493,[주요보유현황] None [업종비중] No._업종_비중(%)_1_정보기술_40.05...,464930,TIGER 글로벌혁신블루칩TOP10
4,0.594885,[주요보유현황] None [업종비중] No._업종_비중(%)_1_정보기술_69.29...,466950,TIGER 글로벌AI액티브
5,0.590206,[주요보유현황] None [업종비중] No._업종_비중(%)_1_정보기술_32.20...,448290,TIGER 미국S&P500TR(H)
6,0.588228,[주요보유현황] None [업종비중] No._업종_비중(%)_1_금융_15.08_2...,458760,TIGER 미국배당+7%프리미엄다우존스
7,0.587449,[주요보유현황] None [업종비중] No._업종_비중(%)_1_산업재_24.94_...,360750,TIGER 미국S&P500
8,0.586792,[주요보유현황] None [업종비중] No._업종_비중(%)_1_정보기술_56.65...,381170,TIGER 미국테크TOP10 INDXX
9,0.586212,[주요보유현황] None [업종비중] No._업종_비중(%)_1_정보기술_38.16...,133690,TIGER 미국나스닥100


## 2) Filter

- 투자성향, 개인/퇴직연금 구분 등 조건에 맞지 않는 ETF는 제외.

In [16]:
candidates = etf_df_fixed[etf_df_fixed['ticker'].isin(candidates_df['ticker'].unique())]

risk_dict = {1: [1, 2, 3, 4, 5, 6],
             2: [2, 3, 4, 5, 6],
             3: [4, 5, 6],
             4: [5, 6],
             5: [6]}

candidates = candidates[candidates['risk'].isin(risk_dict[user_risk_level])]

if user_private_pension:
    candidates = candidates[candidates['private'] == 'True']
if user_retire_pension:
    candidates = candidates[candidates['retire'] == 'True']

filtered_tickers = candidates['ticker'].values
filtered_tickers

array(['341850', '435420', 'A472170'], dtype=object)

In [17]:
selected_df = candidates_df[candidates_df['ticker'].isin(filtered_tickers)]
selected_df

,similarity,sentence,ticker,name
20,0.572662,[주요보유현황] None [업종비중] No._업종_비중(%)_1_etc._27.45...,435420,TIGER 미국나스닥100TR채권혼합Fn
30,0.565796,[주요보유현황] None [업종비중] No._업종_비중(%)_1_정보기술_25.27...,A472170,TIGER미국테크TOP10채권혼합
32,0.564452,[주요보유현황] None [업종비중] No._업종_비중(%)_1_부동산_58.20_...,341850,TIGER 리츠부동산인프라채권TR KIS


## 3) Generate Answer

- 골라진 ETF texts + Text query 를 ClovaChat 에 넣어 답변 생성

In [18]:
selected_sentences = []

for idx, row in selected_df.iterrows():
    sent = f"<BOS> {row['ticker']}: {row['sentence']}"
    selected_sentences.append(sent)

texts = ' <EOS> '.join(selected_sentences)
texts

'<BOS> 435420: [주요보유현황] None [업종비중] No._업종_비중(%)_1_etc._27.45_2_정보기술_15.78_3_커뮤니케이션서비스_4.60_4_자유소비재_3.82_5_헬스케어_1.82_6_필수소비재_1.75_7_산업재_1.17_8_소재_0.42_9_유틸리티_0.35_10_에너지_0.14 [상위10종목] No. 종목명_업종_비중(%)_1_국고채권04125-3312(_18.69_2_국고채권03250-3306(_17.45_3_국고채권03875-2612(_11.84_4_국고채권03250-2903(_11.78_5_국고채권03125-2606(_11.64_6_Microsoft Corp_2.63_7_Apple Inc_2.56_8_NVIDIA Corp_2.38_9_Amazon.com Inc_1.61_10_Broadcom Inc_1.49__ <EOS> <BOS> A472170: [주요보유현황] None [업종비중] No._업종_비중(%)_1_정보기술_25.27_2_etc._12.48_3_커뮤니케이션서비스_9.30_4_자유소비재_5.67 [상위10종목] No. 종목명_업종_비중(%)_1_국고채권04125-3312(_15.93_2_국고채권03250-3306(_14.88_3_국고채권03875-2612(_10.04_4_국고채권03250-2903(_9.99_5_국고채권03125-2606(_9.87_6_Microsoft Corp_7.83_7_Apple Inc_7.74_8_NVIDIA Corp_7.17_9_Alphabet Inc_5.42_10_Amazon.com Inc_4.32__ <EOS> <BOS> 341850: [주요보유현황] None [업종비중] No._업종_비중(%)_1_부동산_58.20_2_etc._27.76_3_금융_11.76 [상위10종목] No. 종목명_업종_비중(%)_1_TIGER 중장기국채_27.77_2_맥쿼리인프라_11.76_3_SK리츠_10.26_4_ESR켄달스퀘어리츠_7.76_5_롯데리츠_6.27_6_제이알글로벌리츠_6.20_7_신한알파리츠

In [21]:
preset_text = [{"role":"system",
                "content": ' '.join(["당신은 증권 전문가입니다.",
                                     "고객은 상장지수펀드 ETF에 관한 질문을 하고, 이는 <Query> 로 입력받습니다.",
                                     "ETF 정보를 담은 데이터베이스를 <DB> 로 입력받습니다.",
                                     "데이터베이스는 운용보고서를 발췌한 것으로, '<BOS> Ticker: [카테고리] 텍스트 <EOS>' 형태 문장으로 이뤄져있습니다."
                                     "반드시 데이터베이스 정보에 근거해 답변하십시오.",
                                     "답변할 때 '데이터베이스' 라는 단어는 사용하지 마십시오.",
                                     "데이터베이스에 없는 정보에 대해서는 '죄송합니다. 그 질문은 답변드릴 수 없습니다.' 라고만 답하십시오."])},
                  {"role":"user","content":f"<Query>: {user_info} </Query> <DB> {texts} </DB>"}]

request_data = {
    'messages': preset_text,
    'topP': 0.8,
    'topK': 0,
    'maxTokens': 2048,
    'temperature': 0.5, 'repeatPenalty': 5.0,
    'stopBefore': [],
    'includeAiFilters': True,
    'seed': 42
}

output = clova_chat.execute(request_data)

id:9facbf2b-b09a-4513-8d6d-5de4076b69a2
event:token
data:{"message":{"role":"assistant","content":"엔"},"index":0,"inputLength":787,"outputLength":1,"stopReason":null}
id:6aa86a5e-cc28-433e-becd-a7b681414bee
event:token
data:{"message":{"role":"assistant","content":"비"},"index":0,"inputLength":787,"outputLength":1,"stopReason":null}
id:ac5330fc-ccdf-4890-a33c-ebf58204fd80
event:token
data:{"message":{"role":"assistant","content":"디아"},"index":0,"inputLength":787,"outputLength":1,"stopReason":null}
id:d12c15bd-282b-4f1e-b626-5449bb0cd9cc
event:token
data:{"message":{"role":"assistant","content":"를"},"index":0,"inputLength":787,"outputLength":1,"stopReason":null}
id:6120be94-a995-4c04-9387-32c38f64f354
event:token
data:{"message":{"role":"assistant","content":" 10"},"index":0,"inputLength":787,"outputLength":1,"stopReason":null}
id:7ce799bd-ff63-459c-9a7e-09f8a75584fc
event:token
data:{"message":{"role":"assistant","content":"%"},"index":0,"inputLength":787,"outputLength":1,"stopReason":n

In [22]:
user_info

'엔비디아 10% 이상 담은 상품 중에서, 수익률 Top 3는?'

In [23]:
import json

answer = json.loads(output.replace('data:', ''))['message']['content']
answer

'엔비디아를 10% 이상 담은 상품 중 수익률 Top 3는 다음과 같습니다. \n\n1. A472170 : 7.17%\n2. 435420 : 2.38%\n\n다만, 문의하신 내용의 순위는 운용보고서 기준일 등에 따라 변동될 수 있음을 참고 부탁드립니다.'